<a href="https://colab.research.google.com/github/shivanikukreti/BigDataA2/blob/master/RTree_Implementation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import time
import sys
import math
import pandas as pd
from tqdm import tqdm

In [0]:
url_rangequeries = 'https://raw.githubusercontent.com/shivanikukreti/BigDataA2/master/100_Queries.txt?token=AM4M6LSUZLTSN5XFKF6W4T25TCAAI'
url_coordinates = 'https://raw.githubusercontent.com/shivanikukreti/BigDataA2/master/Dataset_for_R-Tree.txt?token=AM4M6LUDSF7AONQWNSMGUHS5TCAFQ'

In [0]:
rangequeries = pd.read_csv(url_rangequeries,
                           delimiter=" ",
                           header=None,
                           names=['X1', 'X2', 'Y1', 'Y2'])

In [0]:
rangequeries.head()

,X1,X2,Y1,Y2
0,17840,18840,13971,14971
1,33451,34451,29693,30693
2,791,1791,2515,3515
3,81921,82921,94973,95973
4,75678,76678,53545,54545


In [0]:
coordinates = pd.read_csv(url_coordinates,
                          delimiter=" ",
                          skiprows=1, usecols=[1,2],
                          names=("X-coordinate", "Y-coordinate"))

In [0]:
coordinates.head()

,X-coordinate,Y-coordinate
0,8224,78217
1,68940,46095
2,92607,82845
3,59,99459
4,14140,65521


In [0]:
coordinates.shape

(100000, 2)

In [0]:
B = 3
sequential_result = []

def sequential_query(coordinates, rangequeries):
    start = time.time()
    for idx, query in rangequeries.iterrows():
        rslt = 0
        for index, point in coordinates.iterrows():
            if query['X1'] <= point['X-coordinate'] <= query['X2'] and query['Y1'] <= point['Y-coordinate'] <= query['Y2']:
                rslt += 1
        sequential_result.append(rslt)
    end = time.time()
    total_time = end-start
    return sequential_result, total_time

In [0]:
sequential_result,duration = sequential_query(coordinates, rangequeries)
duration # the time of answering queries by reading the entire dataset sequentially

1182.0544264316559

In [24]:
# average execution time for sequential results
avg_seq = duration/100
avg_seq

11.82054426431656

In [0]:
# building R-tree referencing the code from Week 5 Workshop

class Node(object):
    def __init__(self):
        self.id = 0
        # for internal nodes
        self.child_nodes = []
        # for leaf nodes
        self.data_points = []
        self.parent = None
        self.MBR = {
            'X1': -1,
            'Y1': -1,
            'X2': -1,
            'Y2': -1,
        }

    def perimeter(self):
        # calculating half the perimeter here
        return (self.MBR['X2'] - self.MBR['X1']) + (self.MBR['Y2'] - self.MBR['Y1'])

    def is_underflow(self):
        if self.is_leaf():
            if self.data_points.__len__() < math.ceil(B / 2):
                return True
            else:
                return False
        else:
            if self.child_nodes.__len__() < math.ceil(B / 2):
                return True
            else:
                return False

    def is_overflow(self):
        if self.is_leaf():
            if self.data_points.__len__() > B:
                return True
            else:
                return False
        else:
            if self.child_nodes.__len__() > B:
                return True
            else:
                return False

    def is_root(self):
        if self.parent is None:
            return True
        else:
            return False

    def is_leaf(self):
        if self.child_nodes.__len__() == 0:
            return True
        else:
            return False

class RTree(object):
    def __init__(self):
        self.root = Node()

    def query(self, node, query):
        num = 0
        if node.is_leaf():
            for point in node.data_points:
                if self.is_covered(point, query):
                    num = num + 1
            return num
        else:
            for child in node.child_nodes:
                if self.is_intersect(child, query):
                    num = num + self.query(child, query)
            return num

    def is_intersect(self, node, query):
        # if two mbrs are intersected, then:
        # |center1_x - center2_x| <= length1 / 2 + length2 / 2 and:
        # |center1_y - center2_y| <= width1 / 2 + width2 / 2
        center1_x = (node.MBR['X2'] + node.MBR['X1']) / 2
        center1_y = (node.MBR['Y2'] + node.MBR['Y1']) / 2
        length1 = node.MBR['X2'] - node.MBR['X1']
        width1 = node.MBR['Y2'] - node.MBR['Y1']
        center2_x = (query['X2'] + query['X1']) / 2
        center2_y = (query['Y2'] + query['Y1']) / 2
        length2 = query['X2'] - query['X1']
        width2 = query['Y2'] - query['Y1']
        if abs(center1_x - center2_x) <= length1 / 2 + length2 / 2 and\
                abs(center1_y - center2_y) <= width1 / 2 + width2 / 2:
            return True
        else:
            return False

    def is_covered(self, point, query):
        X1, X2, Y1, Y2 = query['X1'], query['X2'], query['Y1'], query['Y2']
        if X1 <= point['X-coordinate'] <= X2 and Y1 <= point['Y-coordinate'] <= Y2:
            return True
        else:
            return False

    def insert(self, u, p):
        if u.is_leaf():
            self.add_data_point(u, p)
            if u.is_overflow():
                self.handle_overflow(u)
        else:
            v = self.choose_subtree(u, p)
            self.insert(v, p)
            self.update_mbr(v)

    # return the child whose MBR requires the minimum increase in perimeter to cover p
    def choose_subtree(self, u, p):
        if u.is_leaf():
            return u
        else:
            min_increase = sys.maxsize
            best_child = None
            for child in u.child_nodes:
                if min_increase > self.peri_increase(child, p):
                    min_increase = self.peri_increase(child, p)
                    best_child = child
            # return self.choose_subtree(best_child, p)
            return best_child

    def peri_increase(self, node, p):
        # new perimeter - original perimeter = increase of perimeter
        origin_mbr = node.MBR
        X1, X2, Y1, Y2 = origin_mbr['X1'], origin_mbr['X2'], origin_mbr['Y1'], origin_mbr['Y2']
        increase = (max([X1, X2, p['X-coordinate']]) - min([X1, X2, p['X-coordinate']]) +
                    max([Y1, Y2, p['Y-coordinate']]) - min([Y1, Y2, p['Y-coordinate']])) - node.perimeter()
        return increase

    def handle_overflow(self, u):
        u1, u2 = self.split(u)
        # if u is root, create a new root with s1 and s2 as its' children
        if u.is_root():
            new_root = Node()
            self.add_child(new_root, u1)
            self.add_child(new_root, u2)
            self.root = new_root
            self.update_mbr(new_root)
        # if u is not root, delete u, and set s1 and s2 as u's parent's new children
        else:
            w = u.parent
            # copy the information of s1 into u
            w.child_nodes.remove(u)
            self.add_child(w, u1)
            self.add_child(w, u2)
            if w.is_overflow():
                self.handle_overflow(w)
            self.update_mbr(w)

    def split(self, u):
        # split u into s1 and s2
        best_s1 = Node()
        best_s2 = Node()
        best_perimeter = sys.maxsize
        # u is a leaf node
        if u.is_leaf():
            m = u.data_points.__len__()
            # create two different kinds of divides
            divides = [sorted(u.data_points, key=lambda data_point: data_point['X-coordinate']),
                       sorted(u.data_points, key=lambda data_point: data_point['Y-coordinate'])]
            for divide in divides:
                for i in range(math.ceil(0.4 * B), m - math.ceil(0.4 * B) + 1):
                    s1 = Node()
                    s1.data_points = divide[0: i]
                    self.update_mbr(s1)
                    s2 = Node()
                    s2.data_points = divide[i: divide.__len__()]
                    self.update_mbr(s2)
                    if best_perimeter > s1.perimeter() + s2.perimeter():
                        best_perimeter = s1.perimeter() + s2.perimeter()
                        best_s1 = s1
                        best_s2 = s2

        # u is a internal node
        else:
            # create four different kinds of divides
            m = u.child_nodes.__len__()
            divides = [sorted(u.child_nodes, key=lambda child_node: child_node.MBR['X1']),
                       sorted(u.child_nodes, key=lambda child_node: child_node.MBR['X2']),
                       sorted(u.child_nodes, key=lambda child_node: child_node.MBR['Y1']),
                       sorted(u.child_nodes, key=lambda child_node: child_node.MBR['Y2'])]
            for divide in divides:
                for i in range(math.ceil(0.4 * B), m - math.ceil(0.4 * B) + 1):
                    s1 = Node()
                    s1.child_nodes = divide[0: i]
                    self.update_mbr(s1)
                    s2 = Node()
                    s2.child_nodes = divide[i: divide.__len__()]
                    self.update_mbr(s2)
                    if best_perimeter > s1.perimeter() + s2.perimeter():
                        best_perimeter = s1.perimeter() + s2.perimeter()
                        best_s1 = s1
                        best_s2 = s2

        for child in best_s1.child_nodes:
            child.parent = best_s1
        for child in best_s2.child_nodes:
            child.parent = best_s2

        return best_s1, best_s2

    def add_child(self, node, child):
        node.child_nodes.append(child)
        child.parent = node
        # self.update_mbr(node)
        if child.MBR['X1'] < node.MBR['X1']:
            node.MBR['X1'] = child.MBR['X1']
        if child.MBR['X2'] > node.MBR['X2']:
            node.MBR['X2'] = child.MBR['X2']
        if child.MBR['Y1'] < node.MBR['Y1']:
            node.MBR['Y1'] = child.MBR['Y1']
        if child.MBR['Y2'] > node.MBR['Y2']:
            node.MBR['Y2'] = child.MBR['Y2']

    def add_data_point(self, node, data_point):
        node.data_points.append(data_point)
        # self.update_mbr(node)
        if data_point['X-coordinate'] < node.MBR['X1']:
            node.MBR['X1'] = data_point['X-coordinate']
        if data_point['X-coordinate'] > node.MBR['X2']:
            node.MBR['X2'] = data_point['X-coordinate']
        if data_point['Y-coordinate'] < node.MBR['Y1']:
            node.MBR['Y1'] = data_point['Y-coordinate']
        if data_point['Y-coordinate'] > node.MBR['Y2']:
            node.MBR['Y2'] = data_point['Y-coordinate']

    def update_mbr(self, node):
        # print("update_mbr")
        x_list = []
        y_list = []
        if node.is_leaf():
            x_list = [point['X-coordinate'] for point in node.data_points]
            y_list = [point['Y-coordinate'] for point in node.data_points]
        else:
            x_list = [child.MBR['X1'] for child in node.child_nodes] + [child.MBR['X2'] for child in node.child_nodes]
            y_list = [child.MBR['Y1'] for child in node.child_nodes] + [child.MBR['Y2'] for child in node.child_nodes]
        new_mbr = {
            'X1': min(x_list),
            'X2': max(x_list),
            'Y1': min(y_list),
            'Y2': max(y_list)
        }
        node.MBR = new_mbr

In [0]:
rtree = RTree()
coordinates.iloc[0]

X-coordinate     8224
Y-coordinate    78217
Name: 0, dtype: int64

In [0]:
for _,pt in coordinates.iterrows():
    rtree.insert(rtree.root,pt)

In [0]:
rtree_result = list()
start = time.time()
for _,o in rangequeries.iterrows():
    rtree_result.append(rtree.query(rtree.root,o))
end = time.time()
total_time = end-start

In [0]:
# total running time of answering all the 100 queries
total_time

1.2337133884429932

In [20]:
# average time of each query
avg_time = total_time/100
avg_time

0.012337133884429932

In [21]:
# the number of points returned by each query-note
len(rtree_result), rtree_result[:]

(100,
 [15,
  13,
  9,
  8,
  8,
  8,
  12,
  10,
  9,
  11,
  11,
  6,
  6,
  10,
  6,
  8,
  10,
  10,
  11,
  7,
  11,
  9,
  4,
  8,
  11,
  12,
  7,
  13,
  9,
  9,
  8,
  7,
  9,
  9,
  10,
  13,
  9,
  6,
  8,
  13,
  12,
  8,
  8,
  11,
  11,
  10,
  8,
  8,
  12,
  5,
  7,
  4,
  6,
  13,
  9,
  9,
  7,
  10,
  16,
  4,
  11,
  6,
  6,
  16,
  14,
  8,
  11,
  10,
  7,
  16,
  13,
  11,
  10,
  7,
  10,
  9,
  10,
  11,
  6,
  11,
  10,
  8,
  9,
  10,
  20,
  8,
  8,
  12,
  7,
  9,
  5,
  14,
  11,
  5,
  6,
  11,
  11,
  15,
  8,
  14])